In [1]:
# 사용할 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 전 기본 세팅
#한글 깨짐
plt.rc("font", family="Malgun Gothic")

In [2]:
# 파일 불러오기
sample_submission = pd.read_csv('data/sample_submission.csv', index_col='no')
test = pd.read_csv('data/test.csv', index_col='no')
train = pd.read_csv('data/train.csv', index_col='no')

## 예측할 값 : 개인 소득
- income : 50K 초과는 1, 50K 이하는 0 (소득)

### 컬럼 의미
- age - continuous.
- workclass(일 유형) : Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt(일련의 관측 결과를 바탕으로 인구조사국이 부여하는 개인의 가중치): continuous.
- education(교육수준) : Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num(교육수준 번호) : continuous.
- marital-status(결혼 상태) : Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation(직업) : Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship(가족관계) : Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race(인종) : White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex(성별) : Female, Male.
- capital-gain(자본 이익) : continuous.
- capital-loss(자본 손실) : continuous.
- hours-per-week(주당 근무 시간) : continuous.
- native-country(국적) : United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

### value값 띄어쓰기 제거

In [3]:
# 띄어쓰기 제거함
for i in train.columns : 
    if train[f'{i}'].dtype == 'object' :
        train[f'{i}'] = train[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        train[f'{i}'] = train[f'{i}']
        
for i in test.columns : 
    if test[f'{i}'].dtype == 'object' :
        test[f'{i}'] = test[f'{i}'].apply(lambda x: x.replace(' ', '')) # 제거하는 함수
    else :
        test[f'{i}'] = test[f'{i}']

In [4]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', '?',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
cover_title=['Private', 'Self-emp-not-inc', 'Local-gov', 'Never-worked',
       'State-gov', 'Self-emp-inc','Federal-gov','Without-pay','Never-worked']
title_dict = dict(zip(title, cover_title))
title_dict
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] =test['workclass'].map(title_dict)

In [5]:
title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       '?', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
cover_title=['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'United-States', 'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Portugal', 'Columbia', 'Hungary', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
title_dict = dict(zip(title, cover_title))
title_dict
train['native-country'] = train['native-country'].map(title_dict)

test['native-country'] = test['native-country'].map(title_dict)

In [6]:
title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
cover_title=['Machine-op-inspct', 'Other-service', 'Handlers-cleaners',
       'Tech-support', 'Transport-moving', 'Farming-fishing',
       'Prof-specialty', 'Priv-house-serv', 'Adm-clerical',
       'Protective-serv', 'Exec-managerial', '?', 'Craft-repair', 'Sales',
       'Armed-Forces']
title_dict = dict(zip(title, cover_title))
title_dict
train['occupation'] = train['occupation'].map(title_dict)
test['occupation'] = test['occupation'].map(title_dict)

In [7]:
train = train.drop('education',axis=1)
test = test.drop('education',axis=1)

In [8]:
marital_status = ['Divorced', 'Never-married', 'Married-civ-spouse', 'Separated',
       'Married-spouse-absent', 'Widowed', 'Married-AF-spouse']
conver_marital_status = ['not-married', 'not-married', 'married', 'not-married', 'not-married', 'not-married','married']
marital_status_dict=dict(zip(marital_status, conver_marital_status))

train['marital-status']=train['marital-status'].map(marital_status_dict)
test['marital-status']=test['marital-status'].map(marital_status_dict)

In [9]:
train['relationship'].unique()
title=['Not-in-family', 'Husband', 'Unmarried', 'Other-relative',
       'Own-child', 'Wife']
cover_title=['Not-in-family', 'Married', 'Unmarried', 'Other-relative',
       'Own-child', 'Married']
title_dict = dict(zip(title, cover_title))
train['relationship'] = train['relationship'].map(title_dict)

title=['Own-child', 'Wife', 'Husband', 'Not-in-family', 'Unmarried',
       'Other-relative']
cover_title=['Own-child', 'Married', 'Married', 'Not-in-family', 'Unmarried',
       'Other-relative']
title_dict = dict(zip(title, cover_title))
test['relationship'] = test['relationship'].map(title_dict)

In [10]:
"""bins = [0,4,8,12,16] # 범위지정
labels = ['e1', 'e2', 'e3', 'e4'] # 라벨지정
train['education-num'] = pd.cut(train['education-num'], bins=bins, labels = labels)
test['education-num'] = pd.cut(test['education-num'], bins=bins, labels=labels)"""

"bins = [0,4,8,12,16] # 범위지정\nlabels = ['e1', 'e2', 'e3', 'e4'] # 라벨지정\ntrain['education-num'] = pd.cut(train['education-num'], bins=bins, labels = labels)\ntest['education-num'] = pd.cut(test['education-num'], bins=bins, labels=labels)"

In [11]:
"""bins = [i for i in range(0, 101,10)]
labels = ['age' + str(i) for i in range(0, 100,10)] # 라벨지정
train['age'] = pd.cut(train['age'], bins=bins, labels = labels)
test['age'] = pd.cut(test['age'], bins=bins, labels=labels)"""

"bins = [i for i in range(0, 101,10)]\nlabels = ['age' + str(i) for i in range(0, 100,10)] # 라벨지정\ntrain['age'] = pd.cut(train['age'], bins=bins, labels = labels)\ntest['age'] = pd.cut(test['age'], bins=bins, labels=labels)"

In [12]:
title=['Private', 'Self-emp-not-inc', 'Local-gov', 'State-gov',
       'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked']
cover_title=['Private', 'Self-emp', 'gov', 'gov',
       'Self-emp', 'gov', 'beggar', 'beggar']
title_dict = dict(zip(title, cover_title))
train['workclass'] = train['workclass'].map(title_dict)
test['workclass'] = test['workclass'].map(title_dict)

In [13]:
native_country = ['United-States', 'Haiti', 'Mexico', 'Puerto-Rico', 'Philippines',
       'Germany', 'Peru', 'Ecuador', 'Iran', 'Thailand',
       'Dominican-Republic', 'Poland', 'Scotland', 'Italy', 'Jamaica',
       'China', 'Columbia', 'Hungary', 'Portugal', 'Vietnam', 'Taiwan',
       'Canada', 'Hong', 'Guatemala', 'El-Salvador', 'England',
       'Outlying-US(Guam-USVI-etc)', 'India', 'France', 'Cuba', 'Greece',
       'Trinadad&Tobago', 'South', 'Japan', 'Yugoslavia', 'Nicaragua',
       'Ireland', 'Cambodia', 'Laos', 'Honduras', 'Holand-Netherlands']
conver_native_country = ['United-State', 'North-America', 'North-America', 'North-America', 'Asia',
                        'Europe', 'South-America', 'South-America', 'Asia', 'Asia',
                        'North-America', 'Europe', 'Europe', 'Europe', 'North-America',
                        'Asia', 'South-America', 'Europe', 'Europe', 'Asia', 'Asia',
                        'North-America', 'Asia', 'North-America', 'North-America', 'Europe',
                        'North-America', 'Asia', 'Europe', 'North-America', 'Europe',
                        'South-America', 'Asia', 'Asia', 'Europe', 'North-America',
                        'Europe', 'Asia', 'Asia', 'North-America', 'Europe']

native_country_dict = dict(zip(native_country, conver_native_country))
native_country_dict
train['native-country']=train['native-country'].map(native_country_dict)
test['native-country']=test['native-country'].map(native_country_dict)

In [14]:
train['native-country'].value_counts()

United-State     26796
North-America     1282
Asia               657
Europe             458
South-America      112
Name: native-country, dtype: int64

In [15]:
train = train.drop('fnlwgt',axis=1)
test= test.drop('fnlwgt',axis=1)

- 컬럼순서 바꾸기

In [16]:
# 문제/답 분리
X_train = train.loc[:,:'native-country']
y_train = train['income']

In [17]:
X_train = pd.get_dummies(X_train) # 0과 1로만 이루어진 열을 생성
X_test = pd.get_dummies(test)

In [18]:
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
scaler.fit(X_train)
st_transform_X_train =scaler.transform(X_train)
X_train = pd.DataFrame(st_transform_X_train, columns=X_train.columns)
st_transform_X_test =scaler.transform(X_test)
X_test = pd.DataFrame(st_transform_X_test, columns=X_test.columns)

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

그리드 서치

In [20]:
"""Ada_model = AdaBoostClassifier()
Ada_param_grid = {    
    'n_estimators' : [50,150,250],
    'learning_rate' : [0.5,1]
}
Ada_grid = GridSearchCV(Ada_model,
                        Ada_param_grid,
                        cv = 3)
Ada_grid.fit(X_train,y_train)
# best_score_ 메서드
print(Ada_grid.best_score_)
# best_params_ 메서드
print(Ada_grid.best_params_)"""

"Ada_model = AdaBoostClassifier()\nAda_param_grid = {    \n    'n_estimators' : [50,150,250],\n    'learning_rate' : [0.5,1]\n}\nAda_grid = GridSearchCV(Ada_model,\n                        Ada_param_grid,\n                        cv = 3)\nAda_grid.fit(X_train,y_train)\n# best_score_ 메서드\nprint(Ada_grid.best_score_)\n# best_params_ 메서드\nprint(Ada_grid.best_params_)"

In [21]:
"""lgbm = LGBMClassifier()

params = {
    'boosting_type': ['gbdt', 'dart'], # defalt 'gbdt'
    'num_leaves': [20, 31, 50, 70], # default 31
    'max_depth': [-1, 5, 7, 10, 15, 20, 30], # default -1. 끝까지 만드는 것
    'learning_rate': [0.001, 0.01, 0.05, 0.1], # defaut 0.1
}

lgbm_grid_cv = GridSearchCV(lgbm, param_grid=params, scoring='f1', n_jobs=-1, cv=5, verbose=1)
lgbm_grid_cv.fit(X_train,y_train)
print('GridSearchCV 최고 평균 정확도 수치: {:.4f}'.format(lgbm_grid_cv.best_score_))
print('GridSearchCv 최적 하이퍼 파라미터: ', lgbm_grid_cv.best_params_)"""

"lgbm = LGBMClassifier()\n\nparams = {\n    'boosting_type': ['gbdt', 'dart'], # defalt 'gbdt'\n    'num_leaves': [20, 31, 50, 70], # default 31\n    'max_depth': [-1, 5, 7, 10, 15, 20, 30], # default -1. 끝까지 만드는 것\n    'learning_rate': [0.001, 0.01, 0.05, 0.1], # defaut 0.1\n}\n\nlgbm_grid_cv = GridSearchCV(lgbm, param_grid=params, scoring='f1', n_jobs=-1, cv=5, verbose=1)\nlgbm_grid_cv.fit(X_train,y_train)\nprint('GridSearchCV 최고 평균 정확도 수치: {:.4f}'.format(lgbm_grid_cv.best_score_))\nprint('GridSearchCv 최적 하이퍼 파라미터: ', lgbm_grid_cv.best_params_)"

In [22]:
# 그리드서치 바탕으로 하이퍼파라미터 조정
final_Ada_model = AdaBoostClassifier(learning_rate= 1,
                                    n_estimators = 250)
final_Ada_model.fit(X_train,y_train)
pre = final_Ada_model.predict(X_test)

In [23]:
result = cross_val_score(final_Ada_model,
                          X_train,
                          y_train,
                          cv = 3)
result.mean()

0.8665414523485975

In [24]:
final_lgbm_model = LGBMClassifier(learning_rate=0.01,
                                 max_depth = 50,
                                 n_estimators=800,
                                 num_leaves=100)
final_lgbm_model.fit(X_train, y_train)
pre = final_lgbm_model.predict(X_test)


[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002884 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473


In [25]:
result = cross_val_score(final_lgbm_model,
                          X_train,
                          y_train,
                          cv = 3)
result.mean()

[LightGBM] [Info] Number of positive: 4696, number of negative: 14840
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 435
[LightGBM] [Info] Number of data points in the train set: 19536, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240377 -> initscore=-1.150615
[LightGBM] [Info] Start training from score -1.150615
[LightGBM] [Info] Number of positive: 4697, number of negative: 14840
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001878 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 436
[LightGBM] [Info] Number of data points in the train set: 19537, number of used features: 42
[LightGBM] [Info] [bin

0.8662002509120944

In [26]:
final_cb_model = CatBoostClassifier(verbose=100)
final_cb_model.fit(X_train, y_train)
pre = final_cb_model.predict(X_test)


Learning rate set to 0.043583
0:	learn: 0.6507038	total: 143ms	remaining: 2m 22s
100:	learn: 0.2938156	total: 776ms	remaining: 6.91s
200:	learn: 0.2800515	total: 1.41s	remaining: 5.62s
300:	learn: 0.2714029	total: 2.08s	remaining: 4.83s
400:	learn: 0.2646913	total: 2.69s	remaining: 4.02s
500:	learn: 0.2596707	total: 3.29s	remaining: 3.28s
600:	learn: 0.2553938	total: 3.98s	remaining: 2.64s
700:	learn: 0.2519130	total: 4.66s	remaining: 1.99s
800:	learn: 0.2485771	total: 5.27s	remaining: 1.31s
900:	learn: 0.2456461	total: 5.95s	remaining: 653ms
999:	learn: 0.2426842	total: 6.54s	remaining: 0us


In [27]:
result = cross_val_score(final_cb_model,
                          X_train,
                          y_train,
                          cv = 3)
result.mean()

Learning rate set to 0.036654
0:	learn: 0.6556915	total: 5.29ms	remaining: 5.28s
100:	learn: 0.2991872	total: 600ms	remaining: 5.34s
200:	learn: 0.2835026	total: 1.28s	remaining: 5.1s
300:	learn: 0.2736038	total: 1.84s	remaining: 4.27s
400:	learn: 0.2652770	total: 2.38s	remaining: 3.55s
500:	learn: 0.2593173	total: 2.94s	remaining: 2.93s
600:	learn: 0.2550572	total: 3.51s	remaining: 2.33s
700:	learn: 0.2504798	total: 4.05s	remaining: 1.73s
800:	learn: 0.2467878	total: 4.58s	remaining: 1.14s
900:	learn: 0.2429556	total: 5.16s	remaining: 567ms
999:	learn: 0.2393394	total: 5.68s	remaining: 0us
Learning rate set to 0.036655
0:	learn: 0.6567914	total: 5.23ms	remaining: 5.22s
100:	learn: 0.3010942	total: 554ms	remaining: 4.93s
200:	learn: 0.2861169	total: 1.13s	remaining: 4.49s
300:	learn: 0.2761926	total: 1.75s	remaining: 4.06s
400:	learn: 0.2676959	total: 2.31s	remaining: 3.45s
500:	learn: 0.2616937	total: 2.87s	remaining: 2.86s
600:	learn: 0.2569492	total: 3.47s	remaining: 2.3s
700:	learn

0.8704657369566777

In [28]:
final_voting_model = VotingClassifier(
                        estimators=[
                            ('final_Ada_model',final_Ada_model),
                            ('final_lgbm_model',final_lgbm_model),
                            ('final_cb_model',final_cb_model)
                        ],
                        voting='soft'
                        )
# 학습
final_voting_model.fit(X_train,y_train)
# 예측
pre = final_voting_model.predict(X_test)
# 정답제출
result = pd.read_csv('data/sample_submission.csv')
result['income'] = pre
result.to_csv('final_voting_model82.csv', index = False)

[LightGBM] [Info] Number of positive: 7045, number of negative: 22260
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002966 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 29305, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240403 -> initscore=-1.150473
[LightGBM] [Info] Start training from score -1.150473
Learning rate set to 0.043583
0:	learn: 0.6507038	total: 8.04ms	remaining: 8.04s
100:	learn: 0.2938156	total: 737ms	remaining: 6.56s
200:	learn: 0.2800515	total: 1.36s	remaining: 5.41s
300:	learn: 0.2714029	total: 2.05s	remaining: 4.77s
400:	learn: 0.2646913	total: 2.71s	remaining: 4.04s
500:	learn: 0.2596707	total: 3.35s	remaining: 3.33s
600:	learn: 0.2553938	total: 4.03s	remaining: 2.67s
700:	learn: 0.2519130	total: 4.71s	remaining: 2.01s

In [29]:
result = cross_val_score(final_voting_model,
                          X_train,
                          y_train,
                          cv = 3)
result.mean()

[LightGBM] [Info] Number of positive: 4696, number of negative: 14840
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 435
[LightGBM] [Info] Number of data points in the train set: 19536, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240377 -> initscore=-1.150615
[LightGBM] [Info] Start training from score -1.150615
Learning rate set to 0.036654
0:	learn: 0.6556915	total: 6.08ms	remaining: 6.08s
100:	learn: 0.2991872	total: 714ms	remaining: 6.36s
200:	learn: 0.2835026	total: 1.31s	remaining: 5.2s
300:	learn: 0.2736038	total: 1.89s	remaining: 4.4s
400:	learn: 0.2652770	total: 2.48s	remaining: 3.7s
500:	learn: 0.2593173	total: 3.1s	remaining: 3.08s
600:	learn: 0.2550572	total: 3.64s	remaining: 2.42s
700:	learn: 0.2504798	total: 4.19s	remaining: 1.79s
800

0.8694079167899416